In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import pickle

In [ ]:
top200 = pd.read_csv('/content/drive/MyDrive/Stock_Prediction_MLG/ind_nifty200list.csv')

In [ ]:
stock_symbols = top200['Symbol'].tolist()

In [ ]:
stock_symbols = [sym + ".NS" for sym in stock_symbols]
stock_symbols

['360ONE.NS',
 'ABB.NS',
 'ACC.NS',
 'APLAPOLLO.NS',
 'AUBANK.NS',
 'ADANIENSOL.NS',
 'ADANIENT.NS',
 'ADANIGREEN.NS',
 'ADANIPORTS.NS',
 'ADANIPOWER.NS',
 'ATGL.NS',
 'ABCAPITAL.NS',
 'ALKEM.NS',
 'AMBUJACEM.NS',
 'APOLLOHOSP.NS',
 'ASHOKLEY.NS',
 'ASIANPAINT.NS',
 'ASTRAL.NS',
 'AUROPHARMA.NS',
 'DMART.NS',
 'AXISBANK.NS',
 'BSE.NS',
 'BAJAJ-AUTO.NS',
 'BAJFINANCE.NS',
 'BAJAJFINSV.NS',
 'BAJAJHLDNG.NS',
 'BAJAJHFL.NS',
 'BANKBARODA.NS',
 'BANKINDIA.NS',
 'BDL.NS',
 'BEL.NS',
 'BHARATFORG.NS',
 'BHEL.NS',
 'BPCL.NS',
 'BHARTIARTL.NS',
 'BHARTIHEXA.NS',
 'BIOCON.NS',
 'BLUESTARCO.NS',
 'BOSCHLTD.NS',
 'BRITANNIA.NS',
 'CGPOWER.NS',
 'CANBK.NS',
 'CHOLAFIN.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'COCHINSHIP.NS',
 'COFORGE.NS',
 'COLPAL.NS',
 'CONCOR.NS',
 'COROMANDEL.NS',
 'CUMMINSIND.NS',
 'DLF.NS',
 'DABUR.NS',
 'DIVISLAB.NS',
 'DIXON.NS',
 'DRREDDY.NS',
 'DUMMYTATAM.NS',
 'EICHERMOT.NS',
 'ETERNAL.NS',
 'EXIDEIND.NS',
 'NYKAA.NS',
 'FEDERALBNK.NS',
 'FORTIS.NS',
 'GAIL.NS',
 'GMRAIRPORT

In [ ]:
stock_symbols.remove('ITCHOTELS.NS')
stock_symbols.remove('ENRIN.NS')
stock_symbols.remove('DUMMYTATAM.NS')


In [ ]:
len(stock_symbols)

198

##Training Data

In [ ]:
'''
data = yf.download(stock_symbols, start="2020-01-01", end="2025-01-01")['Close']
data.to_csv("unnormalized_nse_stock_data.csv")
'''
data = pd.read_csv('/content/drive/MyDrive/Stock_Prediction_MLG/unnormalized_nse_stock_data.csv')

/tmp/ipython-input-2499426586.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stock_symbols, start="2020-01-01", end="2025-01-01")['Close']
[*********************100%***********************]  198 of 198 completed


In [ ]:
print(f"Number of rows: {data.shape[0]}")
print(f"Number of columns: {data.shape[1]}")

Number of rows: 1238
Number of columns: 198


In [ ]:
data.drop(columns=['MAXHEALTH.NS', 'POWERINDIA.NS', 'SBICARD.NS', 'MAZDOCK.NS'], inplace = True)

In [ ]:
empty_cols = data.columns[data.isna().any()]
print("Empty stocks:", empty_cols)

Empty stocks: Index(['ADANIENSOL.NS', 'BAJAJHFL.NS', 'BHARTIHEXA.NS', 'ETERNAL.NS',
       'HYUNDAI.NS', 'IREDA.NS', 'IRFC.NS', 'JIOFIN.NS', 'KALYANKJIL.NS',
       'LICI.NS', 'LODHA.NS', 'MANKIND.NS', 'NTPCGREEN.NS', 'NYKAA.NS',
       'PAYTM.NS', 'POLICYBZR.NS', 'PREMIERENE.NS', 'SONACOMS.NS', 'SWIGGY.NS',
       'TATATECH.NS', 'UNITDSPR.NS', 'VMM.NS', 'WAAREEENER.NS'],
      dtype='object', name='Ticker')


In [ ]:
data = data.dropna(axis=1, thresh=int(0.80 * len(data)))

In [ ]:
empty_cols = data.columns[data.isna().any()]
print("Empty stocks:", empty_cols)

Empty stocks: Index([], dtype='object', name='Ticker')


In [ ]:
print(f"Number of rows: {data.shape[0]}")
print(f"Number of columns: {data.shape[1]}")

Number of rows: 1238
Number of columns: 171


In [ ]:
def normalize_by_last(prices):
    last = prices.iloc[-1]           # last row, shape: (171,)
    norm_prices = prices / last      # broadcast division
    return norm_prices

In [ ]:
data = normalize_by_last(data)

In [ ]:
data.to_csv('nse_stock_data_cleaned.csv')

##test data

In [ ]:
stock_symbols = list(data.columns)

In [ ]:
#test_data  = yf.download(stock_symbols, start="2025-01-02", end="2025-10-02")['Close']
#test_data.to_csv('unnormalized_test_data.csv')

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/Stock_Prediction_MLG/unnormalized_test_data.csv')

In [ ]:
test_data.drop(columns = ['Date'], inplace = True)

In [ ]:
test_data = normalize_by_last(test_data)

In [ ]:
test_data.to_csv('test_data.csv')

##test and train_15_price.npy

In [ ]:
train_price = data.values

In [ ]:
np.save('train_15_price.npy', train_price)

In [ ]:
test_price = test_data.values

In [ ]:
np.save('test_15_price.npy', test_price)

##test and train_15_eod.npy

In [ ]:
df = pd.concat([data, test_data])

In [ ]:
# Feature engineering
f1 = df.values
f2 = df.rolling(5).mean().fillna(method='bfill')
f3 = df.rolling(10).mean().fillna(method='bfill')
f4 = df.rolling(20).std().fillna(method='bfill')
f5 = df.pct_change().fillna(0)

/tmp/ipython-input-1153530806.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  f2 = df.rolling(5).mean().fillna(method='bfill')
/tmp/ipython-input-1153530806.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  f3 = df.rolling(10).mean().fillna(method='bfill')
/tmp/ipython-input-1153530806.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  f4 = df.rolling(20).std().fillna(method='bfill')


In [ ]:
features = np.stack([f1, f2, f3, f4, f5])
features.shape

(5, 1425, 171)

In [ ]:
def pad_window(w, target=15):
    """Pad the window w from shape (5, k, 171) → (5, 15, 171)."""
    k = w.shape[1]
    if k == target:
        return w

    pad_len = target - k
    last_col = w[:, -1:, :]                      # (5,1,171)
    pad = np.repeat(last_col, pad_len, axis=1)   # (5,pad_len,171)
    return np.concatenate([w, pad], axis=1)

In [ ]:
WINDOW = 15
TRAIN_DAYS = 1238
TEST_DAYS = 187

X_train = np.zeros((TRAIN_DAYS, 5, WINDOW, 171))
X_test  = np.zeros((TEST_DAYS, 5, WINDOW, 171))

# ----- TRAIN -----
for t in range(TRAIN_DAYS):
    w = features[:, t:t+WINDOW, :]    # may be less than 15 for last few? (No — train fits fully)
    w = pad_window(w, WINDOW)
    X_train[t] = w

# ----- TEST -----
for t in range(TEST_DAYS):
    idx = TRAIN_DAYS + t
    w = features[:, idx:idx+WINDOW, :]    # last few slices shorter
    w = pad_window(w, WINDOW)
    X_test[t] = w


In [ ]:
print(X_train.shape)
print(X_test.shape)

(1238, 5, 15, 171)
(187, 5, 15, 171)


In [ ]:
# Save
np.save("train_15_eod.npy", X_train)
np.save("test_15_eod.npy",  X_test)

##test and train_15_label.pkl

In [ ]:
# t-th day returns = (price[t+1] - price[t]) / price[t]
train_returns = (train_price[1:] - train_price[:-1]) / train_price[:-1]
test_returns  = (test_price[1:]  - test_price[:-1])  / test_price[:-1]


In [ ]:
train_15_label = [train_returns[i] for i in range(train_returns.shape[0])]
test_15_label  = [test_returns[i]  for i in range(test_returns.shape[0])]


In [ ]:
train_15_label.append(np.zeros(171))
test_15_label.append(np.zeros(171))

In [ ]:
print(len(train_15_label))
print(len(test_15_label))

1238
187


In [ ]:
import pickle

with open("train_15_label.pkl", "wb") as f:
    pickle.dump(train_15_label, f)

with open("test_15_label.pkl", "wb") as f:
    pickle.dump(test_15_label, f)


##NIFTY_relations.npy


In [ ]:
sector_df = pd.read_csv('/content/drive/MyDrive/Stock_Prediction_MLG/sector_df.csv')

In [ ]:
sector_df = sector_df.drop(columns=['Unnamed: 0'])

In [ ]:
sectors = list(sector_df['Sector'])
sectors = pd.Series(sectors, index=data.columns)

In [ ]:
data = data.sort_index(axis=1)   # ensure consistent stock order
sectors = sectors.loc[data.columns]  # reorder sectors to match

In [ ]:
num_stocks = len(data.columns)
num_stocks

171

In [ ]:
# --- Channel 0: sector adjacency ---
sector_adj = np.zeros((num_stocks, num_stocks))
for i in range(num_stocks):
    for j in range(num_stocks):
        sector_adj[i, j] = 1 if sectors.iloc[i] == sectors.iloc[j] else 0

In [ ]:
np.count_nonzero(sector_adj)

3605

In [ ]:
returns = (data.shift(-1) - data) / data
returns = returns.iloc[:-1]   # drop last row (no t+1 available)

In [ ]:
corr_matrix = returns.corr().values
corr_matrix.shape

(171, 171)

In [ ]:
# corr_matrix is your 171×171 Pearson correlation matrix
corr_matrix = corr_matrix.copy()

# zero diagonal (no self-relations)
np.fill_diagonal(corr_matrix, 0)

# binary adjacency based on |corr| >= 0.4
corr_raw = (np.abs(corr_matrix) >= 0.4).astype(int)

# make symmetric (if needed)
corr_raw = np.maximum(corr_raw, corr_raw.T)

In [ ]:
corr_raw

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
np.count_nonzero(corr_raw)

2938

In [ ]:
# --- Stack them ---
relations = np.stack([sector_adj, corr_raw], axis=2)  # shape: 171×171×2

np.save("NIFTY_relations.npy", relations)

In [ ]:
relations.shape

(171, 171, 2)

In [ ]:
import pickle
s = pickle.load(open("/content/drive/MyDrive/Stock_Prediction_MLG/storage_new.pkl", "rb"))

In [ ]:
s['GCN_3'][0][0]

[(np.int64(484), np.float32(1.3831569), np.float32(0.12084059)),
 (np.int64(757), np.float32(1.3466256), np.float32(-0.03709278)),
 (np.int64(288), np.float32(1.266569), np.float32(-0.009139226)),
 (np.int64(309), np.float32(1.0887027), np.float32(0.16720249)),
 (np.int64(76), np.float32(1.0759883), np.float32(-0.08469674))]